In [7]:
import numpy as np
import revreaddy as rdy
import logging
import datetime
import h5py as h5
import matplotlib.cm as cm

import matplotlib.pyplot as plt
reload(logging)
logging.basicConfig(
     format='%(asctime)s %(levelname)s: %(message)s',
     datefmt='%Y/%m/%d %I:%M:%S',
     level=logging.DEBUG
)

In [11]:
def simulation(boxsize,Diffusion_E,Diffusion_S,Diffusion_P,reactiondistance,intrinsikreaction):
    sim = rdy.Sim()
    sim.kt = 2.437 # kJ/mol -> ~300 K
    sim.timestep = 0.02 # ns
    sim.boxsize = boxsize# nm
    sim.delete_all_particle_types() # also deletes all interactions etc.
    sim.new_type("S", 0.01, Diffusion_S)
    sim.new_type("P", 0.01, Diffusion_P)
    sim.new_type("E", 0.01, Diffusion_E)
    sim.delete_all_reactions()
    sim.new_enzymatic("E+S<->E+S", 0, 1, 2, intrinsikreaction, 0., reactiondistance)
    sim.delete_all_particles()
    for _ in range(300):
         pos = np.random.random(3) * 9.9 - 4.95
         sim.add_particle(pos, 0)
         #pos = np.random.random(3) * 9.9 - 4.95
         #sim.add_particle(pos, 1)
         #pos = np.random.random(3) * 9.9 - 4.95
         #sim.add_particle(pos, 2)
    for _ in range(1):   
        pos = np.random.random(3) * 9.9 - 4.95
        sim.add_particle(pos, 2)
    sim.show_world()
    sim.delete_all_observables()
    d = datetime.datetime.now()
    time=str(getattr(d, 'year','month'))+str(getattr(d,'month'))+str(getattr(d,'day'))+str(getattr(d,'hour'))+str(getattr(d,'minute'))+str(getattr(d,'second'))
    numbers_a_name = time+"numbers_a_test1.h5"
    sim.new_particle_numbers(1, numbers_a_name, 0)
    numbers_b_name = time+"numbers_b_test1.h5"
    mean_e_name = time+"mean_e_test1.h5"
    sim.new_particle_numbers(1, numbers_b_name, 1)
    sim.new_mean_squared_displacement(1,mean_e_name, 2)
    sim.run(1000)
    sim.write_observables_to_file()
    sim.delete_all_observables()
    fa = h5.File(numbers_a_name, 'r')
    fb = h5.File(numbers_b_name, 'r')
    fc = h5.File(mean_e_name, 'r')
    return fa , fb ,fc



In [102]:
def forthandback(boxsize,Diffusion_E,Diffusion_S,Diffusion_P,reactiondistance,intrinsikreactionf,intrinsikreactionb):
    sim = rdy.Sim()
    sim.kt = 2.437 # kJ/mol -> ~300 K
    sim.timestep = 20 # ns
    sim.boxsize = boxsize# nm
    sim.delete_all_particle_types() # also deletes all interactions etc.
    sim.new_type("S", 0.01, Diffusion_S)
    sim.new_type("P", 0.01, Diffusion_P)
    sim.new_type("E", 0.01, Diffusion_E)
    sim.delete_all_reactions()
    sim.new_enzymatic("E+S<->E+S", 0, 1, 2, intrinsikreactionf, intrinsikreactionb, reactiondistance)
    sim.delete_all_particles()
    np.random.seed()
    for _ in range(300):
        
         pos = np.random.random(3) * 9.9 - 4.95
         sim.add_particle(pos, 0)
         #pos = np.random.random(3) * 9.9 - 4.95
         #sim.add_particle(pos, 1)
         #pos = np.random.random(3) * 9.9 - 4.95
         #sim.add_particle(pos, 2)
    for _ in range(1):   
        pos = np.random.random(3) * 9.9 - 4.95
        sim.add_particle(pos, 2)
    sim.show_world()
    sim.delete_all_observables()
    d = datetime.datetime.now()
    time=str(getattr(d, 'year','month'))+str(getattr(d,'month'))+str(getattr(d,'day'))+str(getattr(d,'hour'))+str(getattr(d,'minute'))+str(getattr(d,'second'))
    numbers_a_name = time+"numbers_a_test1.h5"
    sim.new_particle_numbers(1, numbers_a_name, 0)
    numbers_b_name = time+"numbers_b_test1.h5"
    mean_e_name = time+"mean_e_test1.h5"
    sim.new_particle_numbers(1, numbers_b_name, 1)
    sim.new_mean_squared_displacement(1,mean_e_name, 2)
    sim.run(5000)
    sim.write_observables_to_file()
    sim.delete_all_observables()
    fa = h5.File(numbers_a_name, 'r')
    fb = h5.File(numbers_b_name, 'r')
    fc = h5.File(mean_e_name, 'r')
    return fa , fb ,fc


In [103]:
#hier kommt die berechnung des verhältnisses der reactionskoeffizienten und der Concentration vom enzym
def analyt(t,boxsize,Diffusion_E,Diffusion_S,reactiondistance,intrinsikreaction):
    #a=((4*np.pi*reactiondistance**3)*(Diffusion_E+Diffusion_S))/((3*boxsize**3)*(Diffusion_E+Diffusion_P))
    k1=4*np.pi*2*(Diffusion_E+Diffusion_S)*(reactiondistance-np.sqrt((Diffusion_E+Diffusion_S)/intrinsikreaction)+np.tanh(reactiondistance*np.sqrt(intrinsikreaction/(Diffusion_E+Diffusion_S))))
    #t=np.linspace(1,40)
    #vor und rück reaction möglich                                        
    #c=np.exp(-t*a)*fa['particleNumbers'][0]
    #nur vor reaction
    k11=intrinsikreaction*4*np.pi*reactiondistance**3/3
    c=fa['particleNumbers'][0]*np.exp((-t*k1*1)/(boxsize**3))
    c11=fa['particleNumbers'][0]*np.exp((-t*k11*1)/(boxsize**3))

    return t, c,c11

In [104]:
colors = iter(cm.rainbow(np.linspace(0, 1, 4)))
for i in [2,8,20,48]:
    boxsize=10.
    Diffusion_E=i
    Diffusion_S=i
    Diffusion_P=10
    reactiondistance=4
    intrinsikreaction=20
    colornow=next(colors)
    #falist=[]
    #for ii in range(1):
    fa,fb,fc=simulation(boxsize,Diffusion_E,Diffusion_S,Diffusion_P,reactiondistance,intrinsikreaction)
    #    falist.append(fa['particleNumbers'])

    #falist=np.array(falist)
    t,c,c11=analyt(np.array(fa['times']),boxsize,Diffusion_E,Diffusion_S,reactiondistance,intrinsikreaction)
    #paertnum=falist.mean(axis=0)   
    plt.plot(fa['times'], fa['particleNumbers'],color=colornow,label="$D_S+D_E$=$%.2f$" %(2*i))
    plt.plot(t,c,color=colornow)
    
plt.plot(t,c11,color=colornow)
plt.legend()

2016/05/21 07:59:17 INFO: Run with timestep 0.02 and 1000 timesteps
2016/05/21 07:59:30 INFO: Finished after 12.561478 seconds.
2016/05/21 07:59:30 INFO: Run with timestep 0.02 and 1000 timesteps
2016/05/21 07:59:42 INFO: Finished after 12.505313 seconds.
2016/05/21 07:59:42 INFO: Run with timestep 0.02 and 1000 timesteps
2016/05/21 07:59:55 INFO: Finished after 12.335518 seconds.
2016/05/21 07:59:55 INFO: Run with timestep 0.02 and 1000 timesteps
2016/05/21 08:00:07 INFO: Finished after 12.181473 seconds.


In [105]:
plt.show()


In [106]:
#hier kommt die berechnung des verhältnisses der reactionskoeffizienten und der Concentration vom enzym
def analyt1(t,boxsize,Diffusion_E,Diffusion_S,Diffusion_P,reactiondistance,intrinsikreactionf,intrinsikreactionb):
    #a=((4*np.pi*reactiondistance**3)*(Diffusion_E+Diffusion_S))/((3*boxsize**3)*(Diffusion_E+Diffusion_P))
    k1=4*np.pi*(Diffusion_E+Diffusion_S)*(reactiondistance-np.sqrt((Diffusion_E+Diffusion_S)/intrinsikreactionf)+np.tanh(reactiondistance*np.sqrt(intrinsikreactionf/(Diffusion_E+Diffusion_S))))
    k2=4*np.pi*(Diffusion_E+Diffusion_P)*(reactiondistance-np.sqrt((Diffusion_E+Diffusion_P)/intrinsikreactionb)+np.tanh(reactiondistance*np.sqrt(intrinsikreactionb/(Diffusion_E+Diffusion_P))))

    #t=np.linspace(1,40)
    #vor und rück reaction möglich                                        
    #c=np.exp(-t*a)*fa['particleNumbers'][0]
    #nur vor reaction
    #k11=intrinsikreaction*4*np.pi*reactiondistance**3/3
    
    c=fa['particleNumbers'][0]*np.exp((-t*(k1+k2)*30.)/(boxsize**3))+((k2)/(k1+k2))
    c11=((1-((k2)/(k1+k2)))*np.exp((-t*(k1+k2)*1.)/(boxsize**3))+((k2)/(k1+k2)))
    

    return t, c, c11

In [101]:
#basic reaction
colors = iter(cm.rainbow(np.linspace(0, 1, 4)))
for i in [2,8,20,48]:
    boxsize=200.
    Diffusion_E=i
    Diffusion_S=i
    Diffusion_P=i
    reactiondistance=5
    intrinsikreactionf=1
    intrinsikreactionb=20
    colornow=next(colors)
   
    fa,fb,fc=forthandback(boxsize,Diffusion_E,Diffusion_S,Diffusion_P,reactiondistance,intrinsikreactionf,intrinsikreactionb)
    a=[]
    for i in fa['particleNumbers']:
        a.append(fa['particleNumbers'][i])
    a=np.array(a)
    print fa['particleNumbers'][1]
    t,c,c11=analyt1(np.array(fa['times']),boxsize,Diffusion_E,Diffusion_S,Diffusion_P,reactiondistance,intrinsikreactionf,intrinsikreactionb) 
    plt.plot(fa['times'],fa['particleNumbers'],label="$D_S+D_E$=$%.2f$" %(2*i))
    plt.plot(t,c11,color=colornow)
    

plt.legend()
plt.show()

2016/05/21 04:32:49 INFO: Run with timestep 0.5 and 5000 timesteps
2016/05/21 04:33:20 INFO: Finished after 31.189135 seconds.
2016/05/21 04:33:21 INFO: Run with timestep 0.5 and 5000 timesteps
2016/05/21 04:33:52 INFO: Finished after 30.766745 seconds.


278
279

2016/05/21 04:33:53 INFO: Run with timestep 0.5 and 5000 timesteps
2016/05/21 04:34:24 INFO: Finished after 31.262619 seconds.



280

2016/05/21 04:34:25 INFO: Run with timestep 0.5 and 5000 timesteps
2016/05/21 04:34:56 INFO: Finished after 30.794601 seconds.



297


In [68]:
#basic reaction
colors = iter(cm.rainbow(np.linspace(0, 1, 4)))
for i in [2,8,20,48]:
    boxsize=13.
    Diffusion_E=i
    Diffusion_S=i
    Diffusion_P=10
    reactiondistance=4
    intrinsikreactionf=20
    intrinsikreactionb=10
    colornow=next(colors)
    #k1=4*np.pi*2*(Diffusion_E+Diffusion_S)*(reactiondistance-np.sqrt((Diffusion_E+Diffusion_S)/intrinsikreactionf)+np.tanh(reactiondistance*np.sqrt(intrinsikreactionf/(Diffusion_E+Diffusion_S))))
    #k2=4*np.pi*2*(Diffusion_E+Diffusion_P)*(reactiondistance-np.sqrt((Diffusion_E+Diffusion_P)/intrinsikreactionb)+np.tanh(reactiondistance*np.sqrt(intrinsikreactionb/(Diffusion_E+Diffusion_P))))
    #print k1/k2
    fa,fb,fc=forthandback(boxsize,Diffusion_E,Diffusion_S,Diffusion_P,reactiondistance,intrinsikreactionf,intrinsikreactionb)
    t,c,c11=analyt1(np.array(fa['times']),boxsize,Diffusion_E,Diffusion_S,reactiondistance,intrinsikreactionf,intrinsikreactionb) 
    plt.plot(fa['times'], fa['particleNumbers']/fa['particleNumbers'][0],color=colornow,label="$D_S+D_E$=$%.2f$" %(2*i))
    #plt.plot(t,c,color=colornow)
    plt.plot(t,c11,color=colornow)
plt.legend()
plt.show()

2016/05/21 04:01:01 INFO: Run with timestep 0.02 and 1000 timesteps
2016/05/21 04:01:13 INFO: Finished after 12.34485 seconds.
2016/05/21 04:01:13 INFO: Run with timestep 0.02 and 1000 timesteps
2016/05/21 04:01:26 INFO: Finished after 12.484076 seconds.
2016/05/21 04:01:26 INFO: Run with timestep 0.02 and 1000 timesteps
2016/05/21 04:01:38 INFO: Finished after 12.302898 seconds.
2016/05/21 04:01:38 INFO: Run with timestep 0.02 and 1000 timesteps
2016/05/21 04:01:51 INFO: Finished after 12.308786 seconds.


In [54]:
k1=4*np.pi*2*(Diffusion_E+Diffusion_S)*(reactiondistance-np.sqrt((Diffusion_E+Diffusion_S)/intrinsikreactionf)+np.tanh(reactiondistance*np.sqrt(intrinsikreactionf/(Diffusion_E+Diffusion_S))))
k2=4*np.pi*2*(Diffusion_E+Diffusion_S)*(reactiondistance-np.sqrt((Diffusion_E+Diffusion_S)/intrinsikreactionb)+np.tanh(reactiondistance*np.sqrt(intrinsikreactionb/(Diffusion_E+Diffusion_S))))
print k1
print k2

4825.48631591
2412.74315796
